数据集地址：
https://www.kaggle.com/geomack/spotifyclassification/home#data.csv
数据集按0.9和0.1分开，9训练，1检测
用到的数据集是2017首歌的数据集，歌曲拥有的属性分别是
声学，可跳舞性，持续时间ms，能量，工具，键，活跃度，响度，模式，语音，速度，拍号，价数，后三个为target（1为喜欢，0为不喜欢），songtitle，artist


In [1]:
import requests
import os
import zipfile
url = 'https://www.kaggle.com/geomack/spotifyclassification/download'
r = requests.get(url) 
def un_zip(file_name):
    """unzip zip file"""
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(file_name + "_files"):
        pass
    else:
        os.mkdir(file_name + "_files")
    for names in zip_file.namelist():
        zip_file.extract(names,file_name + "_files/")
    zip_file.close()
un_zip('spotifyclassification.zip')
# with open("data.csv", "wb") as code:
#      code.write(r.content)

FileNotFoundError: ignored

In [0]:
import pandas as pd
df = pd.read_csv('./spotifyclassification.zip_files/data.csv')
data = df
data

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
train,test = train_test_split(data,test_size = 0.1, random_state = 2)
train.shape

In [0]:
train = train.drop(['Unnamed: 0','song_title','artist'], axis = 1)
test = test.drop(['Unnamed: 0','song_title','artist'], axis = 1)
y_train = pd.DataFrame(train['target']) 
X_train = train.drop(['target'],axis = 1)
y_test = pd.DataFrame(test['target']) 
X_test = test.drop(['target'],axis = 1)

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

forest = RandomForestClassifier(n_estimators=250)
forest.fit(X_train,y_train.values.ravel())
y_pred_forest = forest.predict(X_test)

accuracyforest = accuracy_score(y_test,y_pred_forest)
accuracyforest

1.	用户可以通过UI或命令行访问，输入这2017首歌的名字，作家，进行查询，如果输入的时候输入错误，在查询结果要出现：“没有此歌曲”，同时，使用机器学习分类方法将歌曲分类为不同的类型（喜欢或不喜欢），并显示出喜好的结果。（任务s1a和s1f）


In [0]:
while(True):
    song_title = input('please input song_title: ')
    artist = input('please input artist: ')    
    # song_title = 'Mask Off'
    # artist = 'Future'
    inputData = data[(data['song_title']==song_title)&(data['artist']==artist)]
    if(len(inputData)==0):
        print('no song found!')
        continue
    inputDataWithId = inputData.drop(['song_title','artist', 'target'], axis = 1)
    inputDataFromat = inputData.drop(['Unnamed: 0','song_title','artist', 'target'], axis = 1)
    for i in range(len(inputDataFromat)):
        pred = forest.predict(inputDataFromat.iloc[i:i+1,:])
        print('------ {} ------'.format(i))
        print(inputDataFromat.iloc[i:i+1,:])
        print('predict: {} true: {}'.format(pred[0], data.at[i, 'target']))
    break
    

1.	根据不同属性建立控制词库，要求输入控制词查询时也能输出前五个结果。（任务s1c）
如：1.instrumentalness的值决定了音频内容，大于0.66的是“talkshow”或“poem”
小于0.66大于0.33的是“hippop”，0.33以下为“Pure music”.

2.tempo为速度，以每分钟心跳数（BPM）为单位，高于140为“fast”，
100-140为“middle”，100以下为“slow”.

3. Valence的值决定了歌曲的感情，0.5以上的对应“开心”或“欢快”，0.5以下对应“悲伤“或”沮丧”.

4.dancebility的值高于0.7则为“舞曲“，低于0.7则为”非舞曲“

5.根据duration_ms，大于200000的为“long song“，小于200000的为”short song“

In [0]:
def instrumentalnessHelper(x):
    if(x > 0.66):
        return 'talkshow'
    if(x > 0.33):
        return 'hippop'
    return 'Pure music'
def tempoHelper(x):
    if(x > 140):
        return 'fast'
    if(x > 100):
        return 'middle'
    return 'slow'
def ValenceHelper(x):
    if x > 0.5:
        return 'happy'
    return 'sad'
def dancebilityHelper(x):
    if(x > 0.7):
        return 'isDance'
    else:
        return 'notDance'
def durationMsHelper(x):
    if(x > 200000):
        return 'long song'
    return 'short song'
data['instrumentalnessName'] = data['instrumentalness'].apply(instrumentalnessHelper)
data['tempoName'] = data['tempo'].apply(tempoHelper)
data['valenceName'] = data['valence'].apply(ValenceHelper)
data['dancebilityName'] = data['danceability'].apply(dancebilityHelper)
data['duration_msName'] = data['duration_ms'].apply(durationMsHelper)
data

In [0]:
print('instrumentalness: talkshow, hippop, Pure music')
print('tempo: fast, middle, slow')
print('valence: happy, sad')
print('dancebility: isDance, notDance')
print('duration_ms: long song, short song')
word = input('please input control word: ')
filterData = data[(data['instrumentalnessName'] == word)|
                 (data['tempoName'] == word)|
                 (data['valenceName'] == word)|
                 (data['dancebilityName'] == word)|
                 (data['duration_msName'] == word)]

filterData.sample(5)

1.	在实现上述功能后，根据余弦相似度或欧氏距离相似度进行歌曲推荐。（任务s1d）


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings(action='ignore')

dataFormat = data.copy()
dataFormat = dataFormat[(dataFormat['song_title']!=song_title)&(dataFormat['artist']!=artist)]


dataFormat = dataFormat.drop(['target', 'instrumentalnessName', 'tempoName',
                'valenceName','dancebilityName','duration_msName'],axis=1)
dataFormatScaler = dataFormat.drop(['Unnamed: 0','song_title','artist'],axis=1)
dataFormat['similarity'] = 1.0
dataFormat['similarity'] = dataFormat['similarity'].astype(float)

mm = MinMaxScaler()
column = dataFormatScaler.columns.tolist()
dataFormatScaler[column]= mm.fit_transform(dataFormatScaler[column])
inputScaler = mm.transform(inputDataFromat.iloc[0:1,:])
similar = []
for i in range(len(dataFormat['similarity'])):
    sim = cosine_similarity(inputScaler, dataFormatScaler.iloc[i:i+1,:])
    similar.append(round(sim[0][0],3))
dataFormat['similarity'] = similar
dataFormat = dataFormat.sort_values(['similarity'], ascending=False)
recommend5 = dataFormat.head(5)

In [0]:
recommend5

1.	另外，当用户选择了一首确定的歌后，要按照两种不同的schema.org的格式输出（rdf和json）。（任务s1e）


In [0]:
id = int(input('please input index of the data(0-2016):'))
# id = 0
chooseData = data.iloc[[id],:]
chooseData

In [0]:
chooseData

In [0]:
chooseData.to_json('{}.json'.format(id))
print('export {}.json successfully'.format(id))

In [0]:
import rdflib
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD
chooseData= chooseData.rename(columns={'Unnamed: 0': 'id'})
columns = chooseData.columns.tolist()
g = rdflib.Graph()
for column in columns:
    cc = column.replace(' ', '_')
    a = rdflib.URIRef("http://www.example.org/{}".format(id))
    b = rdflib.URIRef("http://www.example.org/{}".format(cc))
    if type(chooseData.at[0, column]) == str:
        dd = chooseData.at[0, column].replace(' ', '_')
    else:
        dd = chooseData.at[0, column] 
    c = rdflib.URIRef("http://www.example.org/{}".format(dd))
    g.add((a,b,c))
g.serialize("{}.rdf".format(id))
print('export {}.rdf successfully'.format(id))

In [0]:
data